In [11]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [9]:
load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")

In [10]:
llm = ChatGoogleGenerativeAI(
    model= "gemini-2.0-flash",
    temperature=1.0,
    max_retries=2,
    google_api_key=google_api_key,
)

In [30]:
@tool
def get_conversion_factor(from_currency: str, to_currency: str) -> float:
    """
    Get the conversion factor from one currency to another.
    """
    url = f"https://api.exchangerate-api.com/v4/latest/{from_currency}"
    response = requests.get(url)
    data = response.json()
    
    if to_currency in data['rates']:
        return data['rates'][to_currency]
    else:
        raise ValueError(f"Conversion rate for {to_currency} not found.")

In [56]:
@tool
def convert(amount: float, conversion_rate: float) -> float:
    """
   this function calculate ammount to converted currency using conversion rate.
    """
    return amount * conversion_rate

In [57]:
get_conversion_factor.invoke({'from_currency': 'USD', 'to_currency': 'PKR'})

282.16

In [58]:
convert.invoke({'amount': 100, 'conversion_rate': 280.5})

28050.0

In [59]:
tools = [get_conversion_factor, convert]
llm_with_tools = llm.bind_tools(tools)

In [60]:
llm_with_tools.invoke('what is the current exchange rate from USD to PKR? and convert 100 USD to PKR?')


AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_conversion_factor', 'arguments': '{"to_currency": "PKR", "from_currency": "USD"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-03394a20-292a-4516-a4f0-8165d321678a-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'to_currency': 'PKR', 'from_currency': 'USD'}, 'id': '66015616-3d1c-4293-8df0-a3103a8e5189', 'type': 'tool_call'}], usage_metadata={'input_tokens': 77, 'output_tokens': 14, 'total_tokens': 91, 'input_token_details': {'cache_read': 0}})

In [67]:
query = HumanMessage("what is the current exchange rate from USD to PKR? then using this  convert 100 USD to PKR?")
message = [query]

In [68]:
AI_message = llm_with_tools.invoke(message)

message.append(AI_message)

In [69]:
AI_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'to_currency': 'PKR', 'from_currency': 'USD'},
  'id': 'f9ff1c83-24f1-4bdd-9ac1-55503a02e733',
  'type': 'tool_call'}]

In [72]:
llm_with_tools.invoke("convert 100 USD to PKR? conversion rate is 280.5 also find the conversion rate from USD to PKR?").tool_calls

[{'name': 'convert',
  'args': {'amount': 100.0, 'conversion_rate': 280.5},
  'id': '1972a397-0ac0-486c-8d70-524d4f30911e',
  'type': 'tool_call'},
 {'name': 'get_conversion_factor',
  'args': {'to_currency': 'PKR', 'from_currency': 'USD'},
  'id': '588032e5-9ddd-454c-b1dd-f31855cfd2a0',
  'type': 'tool_call'}]